Elements located via parent_xpath/type[@class="class-name"]

Dependencies

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException

import time

Constants

In [2]:
player_team = ("gyarados,DRAGON_BREATH,AQUA_TAIL,CRUNCH\n"
                "trevenant,SUCKER_PUNCH,SHADOW_BALL,FOUL_PLAY\n"
                "jellicent,HEX,BUBBLE_BEAM,SHADOW_BALL")
opp_team = ("slaking,YAWN,BODY_SLAM\n"
            "porygon-z,LOCK_ON,FRUSTRATION\n"
            "audino,POUND,HYPER_BEAM")
league = '1500 gobattleleague'
difficulty = '0' # 0 is novice 3 is champion

Set up selenium browser, go to pvpoke train page

In [3]:
driver = webdriver.Chrome()
#driver.implicitly_wait(0.3)

In [4]:
driver.get('http://localhost/pvpoke/src/train/')

Set up Player Team

In [5]:
# Your Team Import/Export
driver.find_element_by_xpath('//*[@id="main"]/div[4]/div[1]/div[1]/div/div[1]/button[@class="export-btn"]').click()
time.sleep(0.1)

# Team Text Area
player_team_import = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/textarea[@class="list-text"]')
player_team_import.send_keys(player_team)
time.sleep(0.1)

# Import Team
driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div[@class="button import"]').click()

Choose League

In [6]:
#League & Cup
league_select = driver.find_element_by_xpath('//*[@id="main"]/div[4]/div[2]/div/select[@class="league-cup-select"]')
Select(league_select).select_by_value(league)

Set Difficulty

In [7]:
difficulty_select = driver.find_element_by_xpath('//*[@id="main"]/div[4]/div[2]/div/select[@class="difficulty-select"]')
Select(difficulty_select).select_by_value(difficulty)

Set up Opponent Team

In [8]:
# Team Selection
opp_team_select = driver.find_element_by_xpath('//*[@id="main"]/div[4]/div[2]/div/select[@class="team-method-select"]')
Select(opp_team_select).select_by_value('manual')
time.sleep(0.1)

# Import/Export
driver.find_element_by_xpath('//*[@id="main"]/div[4]/div[2]/div/div[2]/div/div[1]/button[@class="export-btn"]').click()
time.sleep(0.1)

# Text Area
opp_team_import = driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/textarea[@class="list-text"]')
opp_team_import.send_keys(opp_team)
time.sleep(0.1)


driver.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div[@class="button import"]').click()

Start Battle

In [9]:
driver.find_element_by_xpath('//*[@id="main"]/div[5]/button[@class="battle-btn button"]').click()

Interfacing

In [10]:
battle_window_xpath = '//*[@id="main"]/div[@class="section battle"]/div[@class="battle-window"]'

#get_element = lambda xpath : driver.find_element_by_xpath(battle_window_xpath+xpath)
def get_element(lpath):
    classes = lpath.split('/')[1:]
    xpath = ''
    for class_name in classes:
        xpath += '/div[contains(@class, "' + class_name + '")]'
    print('By.XPATH(battle_root + \''+xpath+'\')')
    return driver.find_element_by_xpath(battle_window_xpath+xpath)

In [11]:
battle_window = get_element('')
get_battle_phase = lambda : battle_window.get_attribute('phase')
#print(get_battle_phase())


### class top ###
# other options from here: pokemon types

timer = get_element('/top/timer/text')
get_timer = lambda : int(timer.text if timer.text != '' else -1)

player_top_xpath = '/top/team-indicator left'
pokemon = get_element(player_top_xpath+'/name')
get_pokemon = lambda : pokemon.text
pokemon_cp = get_element(player_top_xpath+'/cp')
get_cp = lambda : int(pokemon_cp.text[3:])
#print(get_cp)

# num of pokemon_remaining's children that don't have 'fainted'
# use get_pokemon_remaining() to get number
pokemon_remaining = get_element(player_top_xpath+'/balls')
get_pokemon_remaining = lambda : len(pokemon_remaining.find_elements_by_xpath('.//div[@class="ball"]'))
#print(get_pokemon_remaining())

shields_remaining = get_element(player_top_xpath+'/shields left')
get_shields_remaining = lambda : int(shields_remaining.text)

opp_top_xpath = '/top/team-indicator right'
opp_pokemon = get_element(opp_top_xpath+'/name')
get_opp_pokemon = lambda : opp_pokemon.text
opp_pokemon_cp = get_element(opp_top_xpath+'/cp')
get_opp_cp = lambda : int(opp_pokemon_cp.text[3:])
#print(get_opp_cp)

opp_pokemon_remaining = get_element(opp_top_xpath+'/balls')
get_opp_pokemon_remaining = lambda : len(opp_pokemon_remaining.find_elements_by_xpath('.//div[@class="ball"]'))
#print(get_opp_pokemon_remaining())

opp_shields_remaining = get_element(opp_top_xpath+'/shields right')
get_opp_shields_remaining = lambda : int(shields_remaining.text)



### class scene ###
# other options from here: turns per fast move
pokemon_hp = get_element('/scene/pokemon-container self/hp bar-back/bar damage')
get_pokemon_hp = lambda : float(pokemon_hp.get_attribute('style')[7:-2])
#print(get_pokemon_hp())

opp_pokemon_hp = get_element('/scene/pokemon-container opponent/hp bar-back/bar damage')
get_opp_pokemon_hp = lambda : float(opp_pokemon_hp.get_attribute('style')[7:-2])
#print(get_opp_pokemon_hp())



### class controls ###
# other options from here: player move names, pause, restart, quit, auto-tap buttons

# if charged_move[i].get_attribute('class') includes 'active', a charged move can be used
charged_move_container = get_element('/controls/move-bars')
charged_move_elems = charged_move_container.find_elements_by_class_name('move-bar')
def get_charged_moves():
    charged_moves = []
    for charged_move in charged_move_elems:
        ready = 'active' in charged_move.get_attribute('class')
        bars = charged_move.find_elements_by_class_name('bar')
        charges = [(66.0 -  float(bar.get_attribute('style')[5:-3]))/66.0 for bar in bars]
        
        charged_moves.append((charged_move, ready, charges))
    return charged_moves

#print(get_charged_moves())



### class shield-window ###
shield_window = get_element('/shield-window')
get_shield_window_closed = lambda : 'closed' in shield_window.get_attribute('class')
shield_button = get_element('/shield-window/container/shield')
no_shield_button = get_element('/shield-window/container/close')



### class charge-window ###
charge_elem = get_element('/charge-window/container/charge')
get_charge = lambda : int(charge_elem.text[0:-1] if charge_elem.text != '' else 0)
#print(get_charge())



### class switch-sidebar ###
# 'pokemon' loses active is fainted
party_container = get_element('/switch-sidebar/container/pokemon-container')
party_elem = party_container.find_elements_by_class_name('pokemon')
def get_party():
    party = []
    for poke in party_elem:
        num = int(poke.get_attribute('index'))
        name = poke.find_element_by_class_name('name').text
        fainted = 'active' not in poke.get_attribute('class')
        # health bar maxes at 75%
        #health = float(poke.find_element_by_class_name('health').text[7:-2])
        party.append((poke, num, name, fainted))
    return party

switch_timer = get_element('/switch-sidebar/container/pokemon-container/pokemon/switch-timer')
get_switch_timer = lambda : int(switch_timer.text if switch_timer.text != '' else 0)
#print(get_party())
#print(get_switch_timer())



### class switch-window ###
switch_container = get_element('/switch-window/container/pokemon-container')
switch_elems = switch_container.find_elements_by_class_name('pokemon')
def get_switch_options():
    party = []
    for poke in switch_elems:
        num = int(poke.get_attribute('index'))
        name = poke.find_element_by_class_name('name').text
        # health bar maxes at 75%
        #health = float(poke.find_element_by_class_name('health').text[7:-2])
        party.append((poke, num, name))
    return party
#print(get_switch_options())



### class animate-message ###
# animate message shows opponents charged move name
animate_message = get_element('/animate-message/text')
get_animate_message = lambda : animate_message.text

#print(get_animate_message())

By.XPATH(battle_root + '')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "timer")]/div[contains(@class, "text")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator left")]/div[contains(@class, "name")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator left")]/div[contains(@class, "cp")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator left")]/div[contains(@class, "balls")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator left")]/div[contains(@class, "shields left")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator right")]/div[contains(@class, "name")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[contains(@class, "team-indicator right")]/div[contains(@class, "cp")]')
By.XPATH(battle_root + '/div[contains(@class, "top")]/div[conta

In [12]:
battle_window = driver.find_element(By.CSS_SELECTOR, '.battle-window')
#battle_window.find_element(By.CSS_SELECTOR, '.shield-window')
timer = '.top .team-indicator.left .name'
print(driver.find_element_by_css_selector(timer).text)

In [13]:
#info
shields = 2
switched = False
previous_phase = ''

while (True):
    time.sleep(0.1)
    battle_phase = get_battle_phase()
    time.sleep(0.1)
    
    # check if game has ended
    if battle_phase == 'game_over_screen':
        break
        
    # suspend_charged_no_shields if user has no more shields
    # check if opponent using charged move
    elif battle_phase == 'suspend_charged_shield':
        shield_button.click()
            
    # check if player using charged move
    elif battle_phase == 'suspend_charged_attack':
        charge = get_charge()
        if charge < 100:
            webdriver.ActionChains(driver).click_and_hold(charge_elem).perform()
            time.sleep(2)
            webdriver.ActionChains(driver).release().perform()
            
        
    # check if first pokemon fainted
    elif battle_phase == 'suspend_switch_self':
        print("Switched should be false:", switched)
        if not switched:
            party = get_switch_options()
            party[0][0].click()
            switched = True
        
    # normal battle phase, can do fast move, charged move, or switch
    elif battle_phase == 'neutral':
        
        # update info
        if previous_phase != 'neutral':
            shields = get_shields_remaining()
            switched = False
            
            print('### Time', get_timer(), 'update ###\n\n'+\
                  '## Player ##\n'+\
                  'Active Pokemon: '+get_pokemon(), get_cp(), 'CP\n'+\
                  'HP Remaining:', get_pokemon_hp(), '%\n'+\
                  'Pokemon Remaining:', get_pokemon_remaining(), '\n'+\
                  #'Shields Remaining:', get_shields_remaining(), '\n\n'+\
                  '## Opponent ##\n'+\
                  'Active Pokemon:', get_opp_pokemon(), get_opp_cp(), 'CP\n'+\
                  'HP Remaining:', get_opp_pokemon_hp(), '%\n'+\
                  'Pokemon Remaining:', get_opp_pokemon_remaining(), '\n'+\
                  #'Shields Remaining:', get_opp_shields_remaining(), '\n\n\n'
                  '')
            
        
        # check if a charged move is ready
        charged_moves = get_charged_moves()
        print("charged move 1 is at", charged_moves[0][2][0])
        print("chaged move 2 is at", charged_moves[1][2][0])
        if charged_moves[1][1]:
            charged_moves[1][0].click()
        elif charged_moves[0][1]:
            charged_moves[0][0].click()
    
        # if no charged move available then fast move
        else:
            battle_window.click()

    
    previous_phase = battle_phase
    time.sleep(0.5)

AttributeError: 'str' object has no attribute 'text'

New Match (Go Back to Player Team)

In [ ]:
driver.find_element_by_xpath('//*[@id="main"]/div[7]/div/div[12]/div/div[2]/div[@class="button new-match"]').click()